In [ ]:
# Importing necessary basic libraries and modules
import classify_detections
from PIL import Image
import numpy as np
import requests
import torch
import os
import cv2
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife.data import transforms as pw_trans
from PytorchWildlife import utils as pw_utils


In [ ]:
# Setting the device to use for computations ('cuda' indicates GPU, "npu" indicates intel NPU)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Initializing the MegaDetectorV5 model for image detection
print(f"Initializing MegaDetectorV5 model for image detection to device {DEVICE}...")
detection_model = pw_detection.MegaDetectorV5(device=DEVICE, pretrained=True)

In [ ]:

# Specifying the path to the target image
url = "https://raw.githubusercontent.com/microsoft/CameraTraps/main/demo/demo_data/imgs/10050028_0.JPG"

# Opening and converting the image to RGB format
img = Image.open(requests.get(url, stream=True).raw).convert("RGB")
orig_image = img

# Save to disk
os.makedirs("demo_output", exist_ok=True)
img_path = os.path.join("demo_output", "detected_camera_image_xyz_timestamp_xxx.jpg")
img.save(img_path)

img = np.array(img)
img.shape, img.dtype


In [ ]:
# Initializing the Yolo-specific transform for the image
transform = pw_trans.MegaDetector_v5_Transform(target_size=detection_model.IMAGE_SIZE,
                                               stride=detection_model.STRIDE)

In [ ]:
# Performing the detection on the single image
results = detection_model.single_image_detection(transform(img), img.shape, img_path)
# Detection results
for key in results:
    print(key, results[key])

In [ ]:
# Saving the detection results 
pw_utils.save_detection_images(results, os.path.join(".","demo_output"), overwrite=False)

Image.open(os.path.join("demo_output", "detected_camera_image_xyz_timestamp_xxx.jpg")).resize((800, 600))

In [ ]:
# Getting ready for classification...

classification_id = 0
classified_animals = []
for xyxy in results["detections"].xyxy:
          # Cropping detection result(s) from original image leveraging detected boxes
          cropped_image = orig_image.crop(xyxy)
          cropped_image.save(os.path.join("demo_output", str(classification_id)+'_cropped_image.jpg'))
            # Performing classification
          classification_result = classify_detections.get_classification(cropped_image)
          print(classification_result)
          classified_animals.append({"id": classification_id,
                                "classification": classification_result,
                                "xyxy": xyxy})
          classification_id = classification_id+1

In [ ]:
# Build classification square
orig_image = np.array(orig_image)
for animal in classified_animals:
    # Classification box attributes
    x1 = int(animal["xyxy"][0])
    y1 = int(animal["xyxy"][1])
    x2 = int(animal["xyxy"][2])
    y2 = int(animal["xyxy"][3])
    color = (255, 0, 0)
    classified_image = cv2.rectangle(orig_image, (x1, y1), (x2, y2), color, 2)
    text_len = len(str(animal["classification"]))
    # Draw black background rectangle to improve text readability
    classified_image = cv2.rectangle(classified_image, (x1, y1), (x2, y1 - 40), color, -1)
    animal["classification"][1] = round(animal["classification"][1], 2)
    # Add label to classification rectangle
    cv2.putText(classified_image, str(animal["classification"]), (x1, y1-10),
                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
    cimg = Image.fromarray(classified_image)
    cimg.save(os.path.join("demo_output",'classified_image.jpg'))
display(cimg)